In [7]:
import numpy as np


In [ ]:
from sklearn.metrics import fbeta_score


In [ ]:
LABEL_TRESHOLD = 0.2

def f2_score(y_true, y_pred):
    return fbeta_score(y_true, y_pred > LABEL_THRESHOLD, beta=2, average='samples')

In [ ]:
true_target = np.loadtxt('true_target.csv')

In [ ]:
predictions = np.loadtxt('predictions.csv')

In [ ]:
filter_array = np.array([0.01,0.02,0.03,0.04,0.05,0.06,0.07,
                        0.08,0.09,0.10,0.11,0.12,0.13,0.14,
                        0.15,0.16,0.17])

In [ ]:
example = np.array([[0.10,0.10,0.10,0.10,0.10,0.10,0.10,
                        0.10,0.10,0.10,0.10,0.10,0.10,0.10,
                        0.10,0.10,0.10],
                   [0.05,0.05,0.05,0.05,0.05,0.05,0.05,
                        0.05,0.05,0.05,0.05,0.05,0.05,0.05,
                        0.05,0.05,0.05],
                   [0.15,0.15,0.15,0.15,0.15,0.15,0.15,
                        0.15,0.15,0.15,0.15,0.15,0.15,0.15,
                        0.15,0.15,0.15]
                   ])

In [ ]:
example > filter_array

In [ ]:
true_target

In [ ]:
def f_opt(threshold):
    return - fbeta_score(true_target, predictions > threshold, beta=2, average='samples')

In [ ]:
f_opt(filter_array)

In [ ]:
import scipy.optimize as optimize

In [ ]:
xopt = optimize.fmin(f_opt, filter_array, xtol=1e-8, disp=True)

In [ ]:
xopt

In [ ]:
filter_array = np.array([-2,30,4,0.0004,0.0895,1.5,89,
                        1000,-35,22,1,0,-0.0987,0.14,
                        0.15,54780,-170])

In [ ]:
xopt = optimize.fmin(f_opt, filter_array, xtol=1e-8, disp=True)

In [ ]:
xopt

In [ ]:
constraints = [(0,1) for i in range(17)]

In [ ]:
constraints

In [ ]:
x1opt = optimize.fmin_l_bfgs_b(f_opt, filter_array, approx_grad = True, bounds=constraints, pgtol=1e-8)

In [ ]:
x1opt

In [ ]:
# Cobyla constraints are build by comparing return value with 0.
# They must be >= 0 or be rejected

def constr_sup0(x):
    return np.min(x)
def constr_inf1(x):
    return 1 - np.max(x)

In [ ]:
x2opt = optimize.fmin_cobyla(f_opt, filter_array, [constr_sup0,constr_inf1])

In [ ]:
x2opt

In [ ]:
f_opt(x2opt)

In [ ]:
filter_array = np.array([0.02,0.02,0.02,0.02,0.02,0.02,0.02,
                        0.02,0.02,0.02,0.02,0.02,0.02,0.02,
                        0.02,0.02,0.02])

In [ ]:
x2opt = optimize.fmin_cobyla(f_opt, filter_array, [constr_sup0,constr_inf1])

In [ ]:
x2opt

In [ ]:
f_opt(x2opt)

In [ ]:
thr_0 = np.array([0.2 for i in range(17)])

In [ ]:
thr_0

# COBYLA is broken and does not respect constraints. coding my own routine

In [10]:
import numpy as np
from sklearn.metrics import fbeta_score


In [28]:
true_target = np.loadtxt('true_target.csv').astype(bool)
predictions = np.loadtxt('predictions.csv')

In [29]:
fbeta_score(true_target, predictions > 0.2, beta=2, average='samples')

0.87536573504345638

In [ ]:
true_target

In [2]:
# True Positive
@jit(nopython=True)
def true_pos(pred_labels, true_labels):
    return np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
 
# True Negative
@jit(nopython=True)
def true_neg(pred_labels, true_labels):
    return np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
 
# False Positive - Type I Error
@jit(nopython=True)
def false_pos(pred_labels, true_labels):
    return np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
 
# False Negative - Type II Error
@jit(nopython=True)
def false_neg(pred_labels, true_labels):
    return np.sum(np.logical_and(pred_labels == 0, true_labels == 1))

In [169]:
@jit(nopython=True)
def precision(pred_labels, true_labels):
    TP = true_pos(pred_labels, true_labels)
    FP = false_pos(pred_labels, true_labels)
    
    # Edge cases True Positives = 0, False negative = 0
    # No predicted labels at all
    # Shouldn't happen all photos must have at least one label
    # We return 0 so that the threshold becomes better
    if TP==0 and FP==0: return 0
    
    return TP / (TP + FP)

@jit(nopython=True)
def recall(pred_labels, true_labels):
    TP = true_pos(pred_labels, true_labels)
    FN = false_neg(pred_labels, true_labels)
    
    # Edge cases True Positives = 0, False negative = 0
    # i.e no label in the true_labels input.
    # Shouldn't happen  all photos have at least one label

    return TP / (TP + FN)

In [170]:
@jit(nopython=True)
def f2_score_macro(pred_labels, true_labels):
    p = precision(pred_labels, true_labels)
    r = recall(pred_labels, true_labels)
    if p == 0 and r == 0: return 0
    return 5 * p * r / (4 * p + r)

In [258]:
from numba import jit, float32, boolean

In [172]:
def f2_score_mean(pred_labels, true_labels):
    return np.mean([f2_score_macro(x,y) for (x,y) in zip(pred_labels,true_labels)])

In [269]:
@jit(nopython=True)
def f2_score_mean(pred_labels, true_labels):
    # Seems like I can't use np.mean on a list within numba ...
    # And I can't use list comprehensions as well
    # so rolling up my own mean proc
    acc = 0
    length=true_labels.shape[0]
    for i in range(length):
        acc += f2_score_macro(pred_labels[i],true_labels[i])
    return acc / length

In [260]:
f2_score_mean(predictions > 0.2, true_target)

0.8753657341003418

In [99]:
fbeta_score(true_target, predictions > 0.2, beta=2, average='samples')

0.87536573504345638

In [100]:
y = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
     [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
     [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])
p = np.array([[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
     [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
     [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
     [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
     [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [175]:
f2_score_mean(p, y)

0.8801891091364776

In [176]:
fbeta_score(y, p, beta=2, average='samples')

0.88018910913647763

In [103]:
for i in range(5):
    print("Predicted: ", p[i])
    print("Truth:     ", y[i])
    print("True Positive: ", true_pos(p[i],y[i]))
    print("True Negative: ", true_neg(p[i],y[i]))
    print("False Positive: ", false_pos(p[i],y[i]))
    print("False Negative: ", false_neg(p[i],y[i]))
    print("Precision: ", precision(p[i],y[i]))
    print("Recall: ", recall(p[i],y[i]))
    print("My F2 score: ", f2_score_macro(p[i],y[i]))
    # print(" Scikit F2 score", fbeta_score(y[i], p[i], beta=2, average='samples'))

Predicted:  [0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0]
Truth:      [0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0]
True Positive:  4
True Negative:  12
False Positive:  1
False Negative:  0
Precision:  0.8
Recall:  1.0
My F2 score:  0.9523809523809523
Predicted:  [1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
Truth:      [1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0]
True Positive:  3
True Negative:  12
False Positive:  1
False Negative:  1
Precision:  0.75
Recall:  0.75
My F2 score:  0.75
Predicted:  [1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1]
Truth:      [1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1]
True Positive:  5
True Negative:  12
False Positive:  0
False Negative:  0
Precision:  1.0
Recall:  1.0
My F2 score:  1.0
Predicted:  [0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0]
Truth:      [0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0]
True Positive:  3
True Negative:  13
False Positive:  0
False Negative:  1
Precision:  1.0
Recall:  0.75
My F2 score:  0.7894736842105263
Predicted:  [0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0]
Truth:      [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0

In [104]:
tp = true_pos(p[0],y[0])

In [105]:
fp = false_pos(p[0],y[0])

In [106]:
tp / (tp + fp)

0.8

In [122]:
true_target = np.loadtxt('true_target.csv').astype(bool)
predictions = np.loadtxt('predictions.csv')

In [123]:
f2_score_mean(predictions>0.2,true_target)

0.8753657350434407

In [124]:
fbeta_score(true_target, predictions > 0.2, beta=2, average='samples')

0.87536573504345638

In [132]:
true_target

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [133]:
predictions > 0.2

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [177]:
def search_best_threshold(p_valid, y_valid, try_all=False, verbose=False):
    p_valid, y_valid = np.array(p_valid), np.array(y_valid)

    best = 0
    best_score = -1
    totry = np.arange(0,1,0.005) if try_all is False else np.unique(p_valid)
    for t in totry:
        score = f2_score_mean(p_valid > t, y_valid)
        if score > best_score:
            best_score = score
            best = t
    if verbose is True: 
        print('Best score: ', round(best_score, 5), ' @ threshold =', best)

    return best

In [178]:
%time search_best_threshold(predictions, true_target)

CPU times: user 2.1 s, sys: 0 ns, total: 2.1 s
Wall time: 2.1 s


0.17999999999999999

In [149]:
def find_f2score_threshold(p_valid, y_valid, try_all=False, verbose=False):
    p_valid, y_valid = np.array(p_valid), np.array(y_valid)

    best = 0
    best_score = -1
    totry = np.arange(0,1,0.005) if try_all is False else np.unique(p_valid)
    for t in totry:
        score = fbeta_score(y_valid, p_valid > t, beta=2, average='samples')
        if score > best_score:
            best_score = score
            best = t
    if verbose is True: 
        print('Best score: ', round(best_score, 5), ' @ threshold =', best)

    return best

In [150]:
%time find_f2score_threshold(predictions, true_target)

/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


CPU times: user 1.93 s, sys: 0 ns, total: 1.93 s
Wall time: 1.93 s


0.17999999999999999

In [181]:
%time f2_score_mean(predictions>0.2,true_target)

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 11.1 ms


0.8753657350434407

In [182]:
%time fbeta_score(true_target, predictions > 0.2, beta=2, average='samples')

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 10.8 ms


0.87536573504345638

In [207]:
from numba import float32

In [279]:
@jit(nopython=True)
def sbt(predictions, true_labels, thresholds = np.arange(0.0,1,0.005,dtype=np.float)):
    ## For each column we search the threshold that maximize the fscore for that column
    ## We are forced to loop on column and rows because we use f2 average of each row
    
    new_f2 = 0.
    best_score = 0.
    best_threshold = np.zeros(predictions.shape[1])
    
    for i in range(true_labels.shape[1]):
        y_proba = predictions[:,i]
        for j in thresholds:
            y_pred = y_proba >= j
            
            # We can maximize f2 for a column
            # On 1D array f2_score_macro and mean have same result
            new_f2 = f2_score_macro(y_pred,true_labels[:,i])
            
            if new_f2 > best_score:
                best_threshold[i] = j
        best_score = 0.
    
    return best_threshold

In [280]:
thr = sbt(predictions, true_target)

In [281]:
thr

array([ 0.995,  0.89 ,  0.79 ,  0.14 ,  0.015,  0.995,  0.99 ,  0.115,
        0.99 ,  0.995,  0.975,  0.995,  0.995,  0.995,  0.08 ,  0.045,
        0.995])

In [277]:
fbeta_score(true_target, predictions > thr, beta=2, average='samples')

/usr/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.23955001672841636

In [274]:
predictions > thr

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [282]:
import nlopt

In [290]:
def f2_thresh_opt(threshold, grad):
    f2_score_mean(predictions>threshold,true_target)

In [291]:
opt = nlopt.opt(nlopt.LN_COBYLA, 17) # dimensionality 17

In [292]:
opt.set_max_objective(f2_thresh_opt)

In [293]:
opt.set_lower_bounds(np.zeros(17))

In [294]:
opt.set_upper_bounds(np.ones(17))

In [296]:
a = np.array([0.2 for i in range(17)])

In [297]:
opt.optimize(a)

ValueError: nlopt invalid argument